<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Optymalizacja-hiperparametrów" data-toc-modified-id="Optymalizacja-hiperparametrów-1">Optymalizacja hiperparametrów</a></span><ul class="toc-item"><li><span><a href="#GridSearchCV" data-toc-modified-id="GridSearchCV-1.1">GridSearchCV</a></span></li><li><span><a href="#RandomizedSearchCV" data-toc-modified-id="RandomizedSearchCV-1.2">RandomizedSearchCV</a></span></li><li><span><a href="#Zadanie" data-toc-modified-id="Zadanie-1.3">Zadanie</a></span></li></ul></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_digits

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV #https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score, accuracy_score, make_scorer

In [ ]:
# Dane digits
digits = load_digits()
X, y = digits.data, digits.target

In [ ]:
#https://scikit-learn.org/stable/modules/cross_validation.html

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)

cross_val_score(clf, X, y, cv=10).mean()

In [ ]:
clf2 = RandomForestClassifier(random_state=0)

cross_val_score(clf2, X, y, cv=10).mean()

# Optymalizacja hiperparametrów

## GridSearchCV

Dokumentacja: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

W metodzie GridSearchCV otrzymamy wyniki z cross walidacji dla wszystkich kombinacji jakie podamy w słowniku parametrów. Metodę tą możemy zasosować dla wszystkich modeli biblioteki sklearn.

In [ ]:
param_grid = {"max_depth": [3, None],
              "max_features": list(range(1,10))
             }
print(param_grid)

In [ ]:
rf_classifier = RandomForestClassifier(random_state=1)

grid_search = GridSearchCV(rf_classifier, param_grid=param_grid, cv=5, verbose=0)
grid_search.fit(X, y)

cv_results_ Zawiera wyniki dla wszystkich kombinacji parametrów z każdej próbki walidacyjnej oraz zagregowane statystyki.

In [ ]:
grid_search.cv_results_

In [ ]:
best_param_no = grid_search.best_index_ 

In [ ]:
best_param_no

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
print('Best params set:', grid_search.best_params_, '(no. {0})'.format(best_param_no))
print('Best params score:', grid_search.best_score_ )

## RandomizedSearchCV

Dokumentacja: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

Podobną metodą zajdowania parametrów jest RandomizedSearchCV. W tym przypadku w przeciwieństwie do GridSearchCV nie będziemy budować modeli dla wszytkich możliwych kombinacji a dla losowo wybranych. W ten sposób może nie znajdziemy tego możliwie najlepszego zestawu ale w dużo krótszym czasie zbliżymy się do niego, dzięki czemu będziemy mogli zawęzić poszukiwania.

In [ ]:
def grid_combination(grid):
    total_params = 1 
    for par in grid:
        total_params  = total_params * len(grid[par])
    return total_params

In [ ]:
param_grid = {"max_depth": [3, None],
              "max_features": list(range(1,6)),
              "n_estimators": [10, 50, 100, 200],
              "min_samples_leaf": [1, 3, 5]}

print(param_grid)
print(f'Liczba wszystkich kombinacji: {grid_combination(param_grid)}')

Score używany w metodach RandomizedSearchCV, GridSearchCV to standardowa miara jakości dla danego modelu sklearn. Przkładowo w przypadku Random Forest jest to accuracy ale możemy podać inną funkcję metryki. Można skorzystać z wszystkich z pakietu sklearn.metris i obiektu make_scorer, który z miary robi odpowiedni scorer.

In [ ]:
from sklearn.metrics import SCORERS

SCORERS.keys()

In [ ]:
random_search = RandomizedSearchCV(rf_classifier,
                                   param_distributions=param_grid,
                                   cv=5,
                                   n_iter=10, #!
                                   verbose=1, 
                                   scoring='accuracy',
                                   #scoring='roc_auc', #multiclass format is not supported - miara dla klasyfikacji dla 2 klas
                                   #scoring=make_scorer(f1_score , average='macro'),
                                   random_state=99)
random_search.fit(X, y)

In [ ]:
random_search.cv_results_

In [ ]:
random_search.best_params_

In [ ]:
random_search.best_score_

In [ ]:
ranks = random_search.cv_results_['rank_test_score']
score = random_search.cv_results_['mean_test_score']

n = 4

for i in range(n):
    idx = np.where(ranks == i + 1)[0][0]
    print(random_search.cv_results_['params'][idx])
    print(score[idx])

## Zadanie

1. Pobierz zbiór danych Titanic zawierający informacje o posażerach titanica. Zadaniem będzie stworzenie możliwie najlepszego modelu Random Forest prognozującego czy dany pasażer przeżył katastrofę:

2. Oczyść ze zbędnych kolumn (PassengerId,Name,Cabin,Ticket) i zdumifikuj odpowiednie zmienne. Uwaga! pamiętaj, że czasem zmienne numeryczne tak naprawdę mają charakter kategoryczny.

3. Podziel zbiór na treningowy i testowy w stosunku 80 - 20. (Użyj random state 0)

4. Znajdź możliwie najlepsze parametry, korzystając z RandomizedSearchCV lub GridSearchCV. (Możesz przetestować działanie obu).

5. Dla najlepszego zestawu hiperparametrów wytrenuj ostateczny model na całym zbiorze treningowym.

6. Policz accuracy na zbiorze testowym i pochwal się wynikiem :)

Przygotowanie danych:

survival - 0 = No, 1 = Yes \
pclas - Ticket clas 1 = 1st, 2 = 2nd, 3 = 3rd \
sex Sex \
Age - Age in years \
sibsp - # of siblings / spouses aboard the Titanic \
parch - # of parents / children aboard the Titanic \
ticket - Ticket number \
fare - Passenger fare \
cabin - Cabin number \
embarked - Port of Embarkation: C = Cherbourg, Q = Queenstown, S = Southampton 

Inna opcja selekcji hiperparametrów:

http://hyperopt.github.io/hyperopt/